In [3]:


%%capture
!pip install keras_tuner
!pip install tensorflow


from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import (
    load_model,
    clone_model
)
import tensorflow as tf


import numpy as np
import os
import sys
from google.colab import userdata

In [4]:
%cd /content
s = userdata.get('ghkey')
with open('./github_token', 'wb') as f:
  f.write(bytes(s, 'utf'))
!rm -rf ./207_final
!git clone --depth 1 https://oauth2:$(cat ./github_token)@github.com/numbersrcool/207_final.git
!rm ./github_token

!ls

/content
Cloning into '207_final'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 45 (delta 6), reused 30 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 106.26 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (6/6), done.
207_final  sample_data


In [5]:
%%capture
b = userdata.get('bucket')
with open('./.bucket', 'wb') as f:
  f.write(bytes(b, 'utf'))

from google.colab import auth
auth.authenticate_user()
!rm -rf data/
!gsutil cp gs://$(cat /content/.bucket)/207_final.zip  /content/
!unzip 207_final.zip




In [6]:
%cd 207_final
!pwd

/content/207_final
/content/207_final


In [7]:
%env MIDS_207_DATA_PATH=../data

from datasets import (
    training,
    validation,
    TRAIN_BATCH_SIZE,
    TRAIN_NUM_BATCHES,
)


env: MIDS_207_DATA_PATH=../data
loading data from ../data...
splitting data into train, val, test...


In [20]:
## Motivation: a CNN model with only 1 but many filters and either average or max pooling.

from tensorflow.keras import layers, models

from models import (
    NUM_CLASSES,
    INPUT_SHAPE,
)

FILTERS = 2048
KERNEL_SIZE = 2
LR = 1e-3

model = models.Sequential()
model.add(layers.Input(INPUT_SHAPE))

model.add(layers.Conv2D(
    filters=FILTERS,
    kernel_size=KERNEL_SIZE,
    activation='relu'
))
model.add(layers.AveragePooling2D(pool_size=(8, 8)))
model.add(layers.Flatten())

model.add(layers.Dense(
    units=256,
    activation='relu'
))

model.add(layers.Dense(NUM_CLASSES, activation='softmax'))



In [21]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=LR
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



In [22]:
tf.keras.backend.clear_session()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [23]:
history = model.fit(training.data,
    epochs=TRAIN_NUM_BATCHES,
    batch_size=TRAIN_BATCH_SIZE,
    validation_data=validation.data,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 420ms/step - accuracy: 0.3955 - loss: 186.2537 - val_accuracy: 0.9688 - val_loss: 0.1246 - learning_rate: 0.0010
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 407ms/step - accuracy: 0.9766 - loss: 0.1089 - val_accuracy: 0.9891 - val_loss: 0.0390 - learning_rate: 0.0010
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 405ms/step - accuracy: 0.9939 - loss: 0.0287 - val_accuracy: 0.9844 - val_loss: 0.0504 - learning_rate: 0.0010
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 404ms/step - accuracy: 0.9846 - loss: 0.0716 - val_accuracy: 0.9812 - val_loss: 0.1069 - learning_rate: 0.0010
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 407ms/step - accuracy: 0.9945 - loss: 0.0417 - val_accuracy: 0.9969 - val_loss: 0.0088 - learning_rate: 5.0000e-04
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 407ms/step - accuracy: 0.9991 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 0.0032 - learning_rate: 5.0000e-04
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 403ms/ste

In [24]:
baseline_model = clone_model(model)
tf.keras.backend.clear_session()
# Unclear if these are mutated from the last call, so we'll just recreate.
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compile from the same config
baseline_model.compile_from_config(model.get_compile_config())


In [25]:
history = baseline_model.fit(training.baseline_data,
    epochs=TRAIN_NUM_BATCHES,
    batch_size=TRAIN_BATCH_SIZE,
    validation_data=validation.baseline_data,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 37s 354ms/step - accuracy: 0.5146 - loss: 58.9740 - val_accuracy: 1.0000 - val_loss: 0.0014 - learning_rate: 2.5000e-04
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 34s 340ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 2.0782e-04 - learning_rate: 2.5000e-04
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 34s 340ms/step - accuracy: 1.0000 - loss: 1.3247e-04 - val_accuracy: 1.0000 - val_loss: 3.5491e-05 - learning_rate: 2.5000e-04
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 34s 339ms/step - accuracy: 1.0000 - loss: 3.2684e-05 - val_accuracy: 1.0000 - val_loss: 1.4039e-05 - learning_rate: 2.5000e-04


In [26]:
baseline_model.save('/content/cnn_C_baseline.keras')
model.save('/content/cnn_C.keras')

In [30]:
#%cd /content
# !gsutil cp cnn_C.keras gs://$(cat /content/.bucket)/
!gsutil cp cnn_C_baseline.keras gs://$(cat /content/.bucket)/

Copying file://cnn_C_baseline.keras [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/1.3 GiB.                                      


In [31]:
!ls -laht /content


total 2.9G
-rw-r--r--  1 root root 1.4G Dec  7 22:22 cnn_C.keras
drwxr-xr-x  1 root root 4.0K Dec  7 22:22 .
-rw-r--r--  1 root root 1.4G Dec  7 22:22 cnn_C_baseline.keras
drwxr-xr-x 37 root root 4.0K Dec  7 20:54 data
-rw-r--r--  1 root root 268M Dec  7 20:54 207_final.zip
drwxr-xr-x  1 root root 4.0K Dec  7 20:54 .config
-rw-r--r--  1 root root   44 Dec  7 20:53 .bucket
drwxr-xr-x  8 root root 4.0K Dec  7 20:53 207_final
drwxr-xr-x  1 root root 4.0K Dec  7 20:33 ..
drwxr-xr-x  1 root root 4.0K Dec  5 14:24 sample_data
